In [28]:
import warnings
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import scipy.spatial.distance as metric
import math
import sklearn.datasets as datasets
import time
from sklearn.preprocessing import normalize
np.set_printoptions(suppress=True)
warnings.filterwarnings("ignore")
from itertools import combinations

In [29]:
iris = datasets.load_iris()
wine = datasets.load_wine()

iris_data = np.array(iris.data)
wine_data = np.array(wine.data)

In [30]:
def calcSSE(data, centroids):
    sum = 0
    for i in data:
        distance = math.inf
        for k in centroids:
            if euc(i, k) < distance:
                distance = euc(i, k)
        sum += distance ** 2

    return sum / len(data)


def euc(A, B):
    # Call to scipy with vector parameters

    return metric.euclidean(A, B)

In [31]:
def rand_cent(ds, k):
    # Number of columns in dataset
    n = np.shape(ds)[1]

    # The centroids
    centroids = np.mat(np.zeros((k, n)))

    # Create random centroids
    for j in range(n):

        min_j = min(ds[:, j])
        range_j = float(max(ds[:, j]) - min_j)
        centroids[:, j] = min_j + range_j * np.random.rand(k, 1)

    # Return centroids as numpy array
    return centroids
def random_datapoints(ds,k):
    index_list = random.sample(range(1,len(ds)),k)
    centroids = ds[index_list]
    return centroids
def naive_sharding(ds, k):
    n = np.shape(ds)[1]

    m = np.shape(ds)[0]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.sum(ds, axis=1)
    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)

    ds.sort(axis=0)
    step = math.floor(m/k)

    vfunc = np.vectorize(_get_mean)

    for j in range(k):
        if j == k-1:
            centroids[j:] = vfunc(np.sum(ds[j*step:, 1:], axis=0), step)
        else:
            centroids[j:] = vfunc(
                np.sum(ds[j*step:(j+1)*step, 1:], axis=0), step)

    return centroids


def mean_sharding(ds, k):
    n = np.shape(ds)[1]

    m = np.shape(ds)[0]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.mean(ds, axis=1)
    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)

    # ds = ds[ds[:, 0].argsort(kind="mergesort")]
    ds.sort(axis=0)

    step = math.floor(m/k)

    vfunc = np.vectorize(_get_mean)

    for j in range(k):
        if j == k-1:
            centroids[j:] = vfunc(np.sum(ds[j*step:, 1:], axis=0), step)
        else:
            centroids[j:] = vfunc(
                np.sum(ds[j*step:(j+1)*step, 1:], axis=0), step)

    return centroids


def median_sharding(ds, k):
    n = np.shape(ds)[1]

    m = np.shape(ds)[0]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.median(ds, axis=1)
    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)

    # ds = ds[ds[:, 0].argsort()]
    ds.sort(axis=0)

    step = math.floor(m/k)

    vfunc = np.vectorize(_get_mean)

    for j in range(k):
        if j == k-1:
            centroids[j:] = vfunc(np.sum(ds[j*step:, 1:], axis=0), step)
        else:
            centroids[j:] = vfunc(
                np.sum(ds[j*step:(j+1)*step, 1:], axis=0), step)

    return centroids

def split_arr(ds, threshold, j):
    if np.size(ds) == 0:
        return None
    min_val = ds[0, 0]

    k = 0
    for i in range(len(ds)):
        if ds[k, 0]-min_val <= threshold:
            # print(k)
            k += 1
        else:
            break

    return [j+k, ds[0:k, :]]

def minmaxsharding(ds, k):

    n = np.shape(ds)[1]

    centroids = np.mat(np.zeros((k, n)))

    composite = np.sum(ds, axis=1)

    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)
    # print(ds)

    # ds = ds[ds[:, 0].argsort()]
    ds.sort(axis=0)

    # print(ds)
    ds_range = np.max(ds[:, 0])-np.min(ds[:, 0])

    threshold = ds_range/k
    prev_arr = split_arr(ds, threshold, 0)

    for j in range(k):
        # print(prev_arr[1])
        centroids[j, :] = np.sum(prev_arr[1][:, 1:], axis=0)/np.shape(prev_arr[1])[0]
        # print(centroids)

        prev_arr = split_arr(ds[prev_arr[0]:, :], threshold, prev_arr[0])
        # print("done")

    return centroids

def l_inf(datas):
    #datas bir numpy arrayi. Parametre olarak np array verilmeli veya alttaki yorum satıtındaki kod çalıştırılmalı:
    #datas=datas.to_numpy() 
    num=len(datas[0])
    for i in range(num):
        m=max(datas[:,i])
        datas[:,i]=datas[:,i]/m
    return datas

def norm_sharding(ds, k):
    
    n = np.shape(ds)[1]
    m = np.shape(ds)[0]

    centroids = np.mat(np.zeros((k, n)))
    
    normds = ds.copy()
    normds = normalize(normds, axis=0, norm='max')
    composite = np.sum(normds, axis=1)
    composite = np.reshape(composite, (len(ds), 1))

    ds = np.append(composite, ds, axis=1)
    ds.sort(axis=0)
    
    step = math.floor(m/k)

    vfunc = np.vectorize(_get_mean)

    for j in range(k):
        if j == k-1:
            centroids[j:] = vfunc(np.sum(ds[j*step:, 1:], axis=0), step)
        else:
            centroids[j:] = vfunc(
                np.sum(ds[j*step:(j+1)*step, 1:], axis=0), step)

    return centroids



In [32]:
def kmeans(ds, k, cent_method):
    global timer_start
    global timer_end
    global total_timer_end
    return_object = {}
    global cents
    global sse
    global iters
    cents = []

    if cent_method == "random":
        timer_start = time.perf_counter()
        cents = rand_cent(ds, k)
        timer_end = time.perf_counter()
    elif cent_method =="random_datapoints":
        timer_start = time.perf_counter()
        cents = random_datapoints(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "naive":
        timer_start = time.perf_counter()
        cents = naive_sharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "mean":
        timer_start = time.perf_counter()
        cents = mean_sharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "median":
        timer_start = time.perf_counter()
        cents = median_sharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "minmax":
        timer_start = time.perf_counter()
        cents = minmaxsharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == "norm_sharding":
        timer_start = time.perf_counter()
        cents = norm_sharding(ds, k)
        timer_end = time.perf_counter()
    elif cent_method == 'near_sharding': 
        timer_start = time.perf_counter()
        cents = near_sharding()
        timer_end = time.perf_counter()
    elif cent_method == 'binary_method': 
        timer_start = time.perf_counter()
        cents = binary_method(ds,k)
        timer_end = time.perf_counter()
    elif cent_method == "corner_method":
        timer_start = time.perf_counter()
        cents = corner_method(ds,k)
        timer_end = time.perf_counter()
    
    km = KMeans(n_clusters=k, init=cents).fit(ds)
    total_timer_end = time.perf_counter()
    iters = km.n_iter_
    cents = km.cluster_centers_
    sse = km.inertia_
    return_object['cents'] = cents
    return_object['time'] = timer_end - timer_start
    return_object['total-time'] = total_timer_end - timer_start
    return_object['sse'] = sse / len(ds)
    return_object['type'] = cent_method
    return_object['iter'] = iters
    return return_object


def _get_mean(sums, step):
    return sums/step

def printResult(datas):
    print("{:<30} {:<30} {:<30} {:<30} {:<30}".format(
        'Type', 'Time', "SSE", "Total Time", "Iter"))
    print('-'*120)
    for d in datas:
        print("{:<30} {:<30} {:<30} {:<30} {:<30}".format(
            d['type'], d['time'], d['sse'], d['total-time'], d['iter']))




In [33]:
def binary_method(ds,k):
  if k == 2:
    centroids = np.array([ds.min(axis=0),ds.max(axis=0)])

  else:
    min_cent = ds.min(axis=0)

    max_cent = ds.max(axis=0)
    
    centroids = np.array([min_cent,max_cent])
    
    diff = (max_cent-min_cent)/(k-1)
    
    for i in range(k-2):
      centroids = np.append(centroids,[min_cent+(i+1)*diff],axis=0)
      
  return centroids

In [34]:
iris_ds = iris.data

In [35]:
def max_diff(ds,k):
    n = np.shape(ds)[1]
    m = np.shape(ds)[0]
    att_sums = sum(ds.T) #Transpose almazsan tüm satırların özelliklerini toplamak yerine 
                         #teker teker özelliklerin tüm örneklerdeki değerlerini topluyor.
    att_sums = np.sort(att_sums)
    diffs = np.zeros((len(att_sums)-1))
    for idx in range(len(att_sums)-1):
        diffs[idx] = att_sums[idx+1] - att_sums[idx]
    
    result = np.argsort(diffs)[-3:]
    return result

In [36]:
def corner_method(ds,k):
    a=2
    if k == 2:
        centroids = np.array([ds.min(axis=0),ds.max(axis=0)])

    else:
        min_cent = ds.min(axis=0)
        max_cent = ds.max(axis=0) 
        centroids = np.array([min_cent,max_cent])
        num_col=ds.shape[1]
        div=math.ceil(num_col/2) #başta yaklaşık yarı yarıya max/min yapacağımız için
        num_cent=k-a # hepsi min ve max dışında bulunması gereken centroid sayısı
        i=0
        while i<num_cent:
            while div>0:
                if(i>=num_cent):
                    break
                num_comb=math.comb(num_col,div) #kombinasyon sayısı
                cols=list(range(num_col)) # column sayısı kadar ardışık sayıdan oluşan array
                combs = list(combinations(cols,div)) #cols un kombinasyonlarını tutan list
                for j in range(num_comb):
                    if(i>=num_cent):
                        break
                    maxes=ds[:,combs[j]].max(axis=0) #kombinasyondaki columnların max değerleri
                    m=list(set(cols)-set(combs[j])) #kalan columnlar
                    mins=ds[:,m].min(axis=0) #kalan columnların min değerleri
                    new_cent=np.zeros(num_col) 
                    vals=np.concatenate((maxes,mins))
                    pos=np.concatenate((combs[j],m))
                    new_cent[pos]=vals
                    #indexlerine göre max ve min değerlerini yerleştirip yeni centroidi belirledik.
                    centroids = np.append(centroids,[new_cent],axis=0)
                    i+=1
                div-=1
    return centroids

In [37]:
df = pd.DataFrame(iris.data)
df = df.to_numpy()
methods = ['random','minmax','median','mean','naive','random_datapoints','norm_sharding','corner_method']#near_sharding
values = [[0 for j in range(5)] for i in range(len(methods))]
repetition = 300
for j in range(len(methods)):
    for i in range(repetition):
        values[j][0] = methods[j]
        a = kmeans(df,3,methods[j])
        values[j][1] += a["time"]
        values[j][2] += a["sse"]
        values[j][3] += a["total-time"]
        values[j][4] += a["iter"]
    for k in range(4):
        values[j][k+1] = values[j][k+1]/repetition
        
print("{:<30} {:<30} {:<30} {:<30} {:<30}".format(
        'Type', 'Time', "SSE", "Total Time", "Iter"))
for i in range(len(values)):
    print("{:<30} {:<30} {:<30} {:<30} {:<30}".format(
            values[i][0], values[i][1], values[i][2], values[i][3], values[i][4]))
    

# printResult([kmeans(df, 3, 'random'), kmeans(df, 3, 'minmax'), 
#              kmeans(df, 3, 'median'), kmeans(df, 3, 'mean'), kmeans(df, 3, 'naive')])

Type                           Time                           SSE                            Total Time                     Iter                          
random                         0.0005559586666913674          0.5631622976979846             0.006922759333330457           8.43                          
minmax                         0.0008019633333196907          0.5256762761743058             0.007901752666663622           4.0                           
median                         0.0007959153333422364          0.5257044388398504             0.004763373000003715           3.0                           
mean                           0.0005801566666635456          0.5257044388398504             0.004145356666660215           3.0                           
naive                          0.0005194063333404604          0.5257044388398504             0.004007592666665308           3.0                           
random_datapoints              7.049433331758337e-05          0.624216

In [38]:
df=wine.data #np arrayi
df=df.astype(float)
df=l_inf(df)

methods = ['random','minmax','median','mean','naive','random_datapoints','norm_sharding','binary_method','corner_method']
values = [[0 for j in range(5)] for i in range(len(methods))]
repetition = 300
for j in range(len(methods)):
    for i in range(repetition):
        values[j][0] = methods[j]
        a = kmeans(df,3,methods[j])
        values[j][1] += a["time"]
        values[j][2] += a["sse"]
        values[j][3] += a["total-time"]
        values[j][4] += a["iter"]
    for k in range(4):
        values[j][k+1] = values[j][k+1]/repetition
        
print("{:<30} {:<30} {:<30} {:<30} {:<30}".format(
        'Type', 'Time', "SSE", "Total Time", "Iter"))
for i in range(len(values)):
    print("{:<30} {:<30} {:<30} {:<30} {:<30}".format(
            values[i][0], values[i][1], values[i][2], values[i][3], values[i][4]))


Type                           Time                           SSE                            Total Time                     Iter                          
random                         0.0014981566666529033          0.15484252797562276            0.006815076999999595           9.396666666666667             
minmax                         0.0008062919999989996          0.15348469815648305            0.00611294633333273            9.0                           
median                         0.0009675486666492361          0.15358078306815307            0.006307566999980736           9.0                           
mean                           0.0007836750000084672          0.15358078306815307            0.007394308666663771           9.0                           
naive                          0.0007711486666721612          0.15358078306815307            0.007174573666667736           9.0                           
random_datapoints              8.846266665689957e-05          0.153611